In [ ]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df_courses = pd.read_json('./data/course_sample_overviews.json')

In [ ]:
with open('./data/courses_sample.json', "r") as f:
    reviews = json.load(f)

In [ ]:
def get_reviews(row):
    course_code = row[0]
    revs = reviews[course_code]
    revs = [rev['review'] for rev in revs]
    return np.array(revs)

In [ ]:
df_courses['reviews'] = df_courses.apply(get_reviews, axis=1)

In [ ]:
def good_course(row, col_name='liked', threshold=50):
    percent = row[col_name]
    percent = percent.replace('%', '')
    if not percent.isnumeric():
        # If no rating, return 0 (not a good course)
        return 0
    percent = int(percent)
    if percent >  threshold:
        return 1
    return 0

In [ ]:
df_courses['num_reviews'] = df_courses.apply(lambda x: len(x['reviews']), axis=1)

In [ ]:
def tmp(row):
    num_reviews = row['num_reviews']
    review_likes = np.random.choice([0, 1], size=(num_reviews,), p=[1./3, 2./3])
    return review_likes

In [ ]:
df_courses['review_likes'] = df_courses.apply(tmp, axis=1)

In [ ]:
df_courses['good_course'] = df_courses.apply(good_course, axis=1)

In [ ]:
subset = df_courses.loc[(df_courses['num_ratings'] >= 5) & (df_courses['num_reviews'] >= 5)]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(list(np.concatenate(subset['reviews'].values).flat), list(np.concatenate(subset['review_likes'].values).flat), test_size=0.33, random_state=6)

## HF Trainer

In [ ]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
from torch import nn
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

In [ ]:
dataset = {
    'train': Dataset.from_dict({'label':y_train,'text':X_train}),
    'test': Dataset.from_dict({'label':y_test,'text':X_test})
}

dataset = DatasetDict(dataset)

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize_text(texts):
    return tokenizer(texts["text"], truncation=True, max_length=512)

In [ ]:
dataset = dataset.map(tokenize_text, batched=True)

In [ ]:
id2label = {0: "bad course", 1: "good course"}
label2id = {"bad course": 0, "good course": 1}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    return {"f1": f1}

In [ ]:
batch_size = 64
logging_steps = len(X_train) // batch_size
output_dir = "hf_trainer"
training_args = TrainingArguments(
    output_dir=output_dir,
     num_train_epochs=5,
     learning_rate=2e-5,
     per_device_train_batch_size=batch_size,
     per_device_eval_batch_size=batch_size,
     weight_decay=0.01,
     evaluation_strategy="epoch",
     logging_steps=logging_steps,
     #p16=True,
     push_to_hub=False
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=tokenizer
)

In [ ]:
trainer.train()